In [1]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow import keras
print(keras.__version__)

2.1.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1
2.2.4-tf


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 2)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:

training_data = np.load('training_data.npy', allow_pickle=True)
x_train = []
y_train = []

for x in range(len(training_data)):
    x_train.append(training_data[x][0])
    y_train.append(training_data[x][1][:3])
    
x_train = np.array(x_train, dtype=np.float16)
y_train = np.array(y_train, dtype=np.float16)



In [3]:
print(x_train[0][0][0][0])
for a in range(len(x_train)):
    for b in range(len(x_train[a])):
        for c in range(len(x_train[a][b])):
            x_train[a][b][c]  =  x_train[a][b][c] / 255

print(x_train[0][0][0][0])

[255. 255. 255.]
[1. 1. 1.]


In [4]:
from keras.models import Sequential
from keras.layers import TimeDistributed, LSTM, Flatten, Dense, InputLayer, MaxPooling2D, Dropout, Activation, Embedding, GRU, ConvLSTM2D
from keras.layers.convolutional import Convolution2D
from keras import optimizers
from keras.models import load_model
from keras import initializers

model = Sequential()


model.add(TimeDistributed(Convolution2D(32, (4,4), data_format='channels_last'), input_shape=(3, 150, 200, 3)))
model.add(TimeDistributed(Activation('relu')))
print(model.output_shape)

model.add(TimeDistributed(Convolution2D(32, (4,4), data_format='channels_last')))
model.add(TimeDistributed(Activation('relu')))
print(model.output_shape)

model.add(TimeDistributed(MaxPooling2D(pool_size=(5, 5), data_format='channels_last')))
model.add(TimeDistributed(Dropout(0.25)))
print(model.output_shape)

model.add(TimeDistributed(Convolution2D(16, (3,3), data_format='channels_last')))
model.add(TimeDistributed(Activation('relu')))
print(model.output_shape)


model.add(TimeDistributed(MaxPooling2D(pool_size=(5, 5), data_format='channels_last')))
model.add(TimeDistributed(Dropout(0.25)))
print(model.output_shape)

model.add(TimeDistributed(Flatten()))
print(model.output_shape)

model.add(LSTM(256, kernel_initializer=initializers.RandomNormal(stddev=0.001))) #128
model.add(Dropout(0.25))
print(model.output_shape)

model.add(Dense(100))
print(model.output_shape)

model.add(Dense(80))
print(model.output_shape)

model.add(Dense(40))
print(model.output_shape)

model.add(Dense(3, activation='sigmoid'))
print(model.output_shape)

model.summary()


Using TensorFlow backend.


(None, 3, 147, 197, 32)
(None, 3, 144, 194, 32)
(None, 3, 28, 38, 32)
(None, 3, 26, 36, 16)
(None, 3, 5, 7, 16)
(None, 3, 560)
(None, 256)
(None, 100)
(None, 80)
(None, 40)
(None, 3)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 3, 147, 197, 32)   1568      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 3, 147, 197, 32)   0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 3, 144, 194, 32)   16416     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 3, 144, 194, 32)   0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 3, 28, 38, 32)     0         
_________________________________________________________________
tim

In [5]:
opt = optimizers.RMSprop(lr=0.001)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])

In [6]:
model.fit(x_train,y_train,
          batch_size=20,
          epochs=5,
          verbose=1)



Epoch 1/5
6918/6918 [==============================] - 40s 6ms/step - loss: 0.1368 - accuracy: 0.6996
Epoch 2/5
6918/6918 [==============================] - 35s 5ms/step - loss: 0.0861 - accuracy: 0.8290
Epoch 3/5
6918/6918 [==============================] - 35s 5ms/step - loss: 0.0689 - accuracy: 0.8648
Epoch 4/5
6918/6918 [==============================] - 35s 5ms/step - loss: 0.0537 - accuracy: 0.8948
Epoch 5/5
6918/6918 [==============================] - 35s 5ms/step - loss: 0.0398 - accuracy: 0.9243


In [7]:
model.save("model_awd.h5")

In [9]:
print(training_data[427][1])

[0, 1, 0, 0, 0, 0]
